In [1]:
%cd ..

/Users/floydluo/Library/CloudStorage/OneDrive-JohnsHopkins/000Projects/0000-Infrastructure/0000-RecFld/RecFldGrn


# Load Data


## Load Raw

In [2]:
import os 
import pandas as pd

RecName = 'Smoking'
folder = 'data/Simulations/'
raw_file_path =  os.path.join(folder, RecName) + '.csv'
print(raw_file_path)


data/Simulations/Smoking.csv


In [3]:

# load all records
df = pd.read_csv(raw_file_path)
df

,PID,ECID,RID,DT,V
0,P0,EC23,15471,2023-03-19,B
1,P0,EC142,21622,2023-03-20,D
2,P0,EC113,19101,2023-03-23,A
3,P0,EC166,22867,2023-03-26,B
4,P0,EC91,19104,2023-03-27,D
...,...,...,...,...,...
96,P5,EC129,20285,2023-04-16,B
97,P6,EC92,14290,2023-03-19,D
98,P6,EC123,17942,2023-03-21,A
99,P6,EC15,17943,2023-03-28,B


## Select Columns 

In [4]:
parent_ids = ['PID','ECID']
focal_ids = ['RID']


selected_cols = ['V', 'DT']

df = df[parent_ids + focal_ids + selected_cols]
df

,PID,ECID,RID,V,DT
0,P0,EC23,15471,B,2023-03-19
1,P0,EC142,21622,D,2023-03-20
2,P0,EC113,19101,A,2023-03-23
3,P0,EC166,22867,B,2023-03-26
4,P0,EC91,19104,D,2023-03-27
...,...,...,...,...,...
96,P5,EC129,20285,B,2023-04-16
97,P6,EC92,14290,D,2023-03-19
98,P6,EC123,17942,A,2023-03-21
99,P6,EC15,17943,B,2023-03-28


## Basic Filtering 

In [5]:
# nothing

## DF Rec Raw

In [6]:
df_RecRaw = df
df_RecRaw

,PID,ECID,RID,V,DT
0,P0,EC23,15471,B,2023-03-19
1,P0,EC142,21622,D,2023-03-20
2,P0,EC113,19101,A,2023-03-23
3,P0,EC166,22867,B,2023-03-26
4,P0,EC91,19104,D,2023-03-27
...,...,...,...,...,...
96,P5,EC129,20285,B,2023-04-16
97,P6,EC92,14290,D,2023-03-19
98,P6,EC123,17942,A,2023-03-21
99,P6,EC15,17943,B,2023-03-28


# Fields in Records

In [7]:
df = df_RecRaw.copy()

## Field: ID

In [8]:
RecName = 'Smoking'
RID = RecName + 'ID'
df = df.rename(columns = {'RID': RID})
assert df[RID].value_counts().max() == 1
df.head()

,PID,ECID,SmokingID,V,DT
0,P0,EC23,15471,B,2023-03-19
1,P0,EC142,21622,D,2023-03-20
2,P0,EC113,19101,A,2023-03-23
3,P0,EC166,22867,B,2023-03-26
4,P0,EC91,19104,D,2023-03-27


## Field: V

In [9]:
df['V']

0      B
1      D
2      A
3      B
4      D
      ..
96     B
97     D
98     A
99     B
100    A
Name: V, Length: 101, dtype: object

## Field: DT

In [10]:
df['DT']

0      2023-03-19
1      2023-03-20
2      2023-03-23
3      2023-03-26
4      2023-03-27
          ...    
96     2023-04-16
97     2023-03-19
98     2023-03-21
99     2023-03-28
100    2023-04-04
Name: DT, Length: 101, dtype: object

## Call Parents

In [11]:
def load_df_data_from_folder(data_folder, IDName):
    file_list = os.listdir(data_folder)
    # file_lis
    df = pd.concat([pd.read_pickle(os.path.join(data_folder, i)) for i in file_list if '.p' in i]).reset_index(drop = True)
    df = df.sort_values(IDName).reset_index(drop = True)
    return df# .shape

fullrec_folder = 'data/ProcData/RecFolder/EC'
df_Pat = load_df_data_from_folder(fullrec_folder, 'ECID')[['ECID','PID']]
print(df_Pat.shape)
df_Pat.head()
print(df_Pat)


df = pd.merge(df_Pat, df_RecRaw, on = parent_ids, how = 'left')
df

(225, 2)
      ECID PID
0      EC0  P1
1      EC1  P3
2     EC10  P4
3    EC100  P5
4    EC101  P3
..     ...  ..
220   EC95  P2
221   EC96  P5
222   EC97  P5
223   EC98  P3
224   EC99  P5

[225 rows x 2 columns]


,ECID,PID,RID,V,DT
0,EC0,P1,NaN,NaN,NaN
1,EC1,P3,NaN,NaN,NaN
2,EC10,P4,16453.0,D,2023-04-15
3,EC100,P5,17833.0,C,2023-03-19
4,EC101,P3,20384.0,A,2023-04-16
...,...,...,...,...,...
226,EC95,P2,NaN,NaN,NaN
227,EC96,P5,17834.0,B,2023-04-02
228,EC97,P5,21524.0,C,2023-04-14
229,EC98,P3,15447.0,A,2023-04-13


In [12]:
import numpy as np

d = df
for col in focal_ids: 
    d.loc[d[col].isnull(), col] = ['Missing' + str(i) for i in np.ones(d[col].isnull().sum()).cumsum()]
    # d[col] = d[col].astype(int)
    assert len(d[col]) == d[col].nunique()
    
df = d
df

,ECID,PID,RID,V,DT
0,EC0,P1,Missing1.0,NaN,NaN
1,EC1,P3,Missing2.0,NaN,NaN
2,EC10,P4,16453.0,D,2023-04-15
3,EC100,P5,17833.0,C,2023-03-19
4,EC101,P3,20384.0,A,2023-04-16
...,...,...,...,...,...
226,EC95,P2,Missing130.0,NaN,NaN
227,EC96,P5,17834.0,B,2023-04-02
228,EC97,P5,21524.0,C,2023-04-14
229,EC98,P3,15447.0,A,2023-04-13


## Pin Down

In [13]:
df_proc = df[['PID', 'ECID', RID, 'V','DT']]
df_proc

,PID,ECID,RID,V,DT
0,P1,EC0,Missing1.0,NaN,NaN
1,P3,EC1,Missing2.0,NaN,NaN
2,P4,EC10,16453.0,D,2023-04-15
3,P5,EC100,17833.0,C,2023-03-19
4,P3,EC101,20384.0,A,2023-04-16
...,...,...,...,...,...
226,P2,EC95,Missing130.0,NaN,NaN
227,P5,EC96,17834.0,B,2023-04-02
228,P5,EC97,21524.0,C,2023-04-14
229,P3,EC98,15447.0,A,2023-04-13


# Save and Load

## Save

In [14]:
from recfldgrn.datapoint import convert_PID_to_PIDgroup
from recfldgrn.datapoint import RANGE_SIZE, write_df_to_folders, load_df_data_from_folder
RANGE_SIZE = 10000; IDNAME = 'PID'

In [15]:
# the folder to save the Record.
###############
RecName = RecName
df = df_proc
###############

rec_folder = 'Data/ProcData/RecFolder'
data_folder = os.path.join(rec_folder, RecName)
print(data_folder, '<----- fullrec_folder to save the records')

if not os.path.exists(data_folder): 
    os.makedirs(data_folder)

write_df_to_folders(RecName, data_folder, df, IDNAME)

Data/ProcData/RecFolder/Smoking <----- fullrec_folder to save the records


## Load

In [16]:
from recfldgrn.datapoint import load_df_data_with_RecName
df = load_df_data_with_RecName(rec_folder, RecName)
print(df.shape)
df.head()

(231, 5)


,PID,ECID,RID,V,DT
0,P1,EC0,Missing1.0,NaN,NaN
1,P3,EC1,Missing2.0,NaN,NaN
2,P4,EC10,16453.0,D,2023-04-15
3,P5,EC100,17833.0,C,2023-03-19
4,P3,EC101,20384.0,A,2023-04-16


## Load One Sample

In [17]:
from recfldgrn.datapoint import DataPoint
####################
PIDValue = 'P5'  
######################
Pat = DataPoint(IDNAME, PIDValue, rec_folder, RANGE_SIZE)
Pat

<PID [P5]>

In [18]:
Pat.get_df_rec(RecName)

,PID,ECID,RID,V,DT
0,P5,EC100,17833.0,C,2023-03-19
1,P5,EC103,Missing3.0,NaN,NaN
2,P5,EC104,Missing4.0,NaN,NaN
3,P5,EC105,Missing5.0,NaN,NaN
4,P5,EC11,Missing8.0,NaN,NaN
5,P5,EC129,20285.0,B,2023-04-16
6,P5,EC133,22749.0,B,2023-04-02
7,P5,EC134,Missing21.0,NaN,NaN
8,P5,EC135,22751.0,D,2023-04-07
9,P5,EC137,16627.0,D,2023-03-29
